In [1]:
import warnings
import datetime
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
# plt.rcParams['figure.figsize'] = 12,40

In [2]:
dfMarketData = pd.read_csv('WDODATA.csv','\t')
dfMarketData.columns = ['Date','Time','Open','High','Low','Close','TickVol','Vol','Spread']
dfMarketData['Datetime'] = dfMarketData['Date'].str.replace('.','-') + ' ' + dfMarketData['Time']
dfMarketData = dfMarketData[['Datetime','Open','High','Low','Close']]
dfMarketData['Returns'] = dfMarketData['Close'].pct_change()
dfMarketData['Datetime'] = pd.to_datetime(dfMarketData['Datetime'])
dfMarketData = dfMarketData.set_index('Datetime')
dfMarketData = dfMarketData.sort_index()
dfMarketData.tail()

,Open,High,Low,Close,Returns
Datetime,,,,,
2023-04-06 13:55:00,5082.0,5083.0,5081.5,5082.0,0.000098
2023-04-06 13:56:00,5081.5,5082.5,5081.5,5082.5,0.000098
2023-04-06 13:57:00,5082.0,5084.0,5082.0,5083.5,0.000197
2023-04-06 13:58:00,5083.5,5083.5,5081.5,5081.5,-0.000393
2023-04-06 13:59:00,5081.5,5082.0,5081.5,5081.5,0.000000


In [3]:
dfEventsData = pd.read_csv('EventsData.csv')
dfEventsFromTo = pd.read_csv('EventsFromTo.csv')
dfEventsFromTo=dfEventsFromTo.set_index('EventID').T
dfEventsData['ReleaseTime'] = pd.to_datetime(dfEventsData['Datetime'])
dfEventsData['PercentChng'] = dfEventsData['ActualValue'].pct_change()
dfEventsData['DummyChng'] = 0
dfEventsData.loc[dfEventsData['PercentChng']>0, 'DummyChng'] = 1
dfEventsData.loc[dfEventsData['PercentChng']<0, 'DummyChng'] = -1
dfEventsData['Date'] = dfEventsData['ReleaseTime'].apply(lambda x: datetime.datetime.strptime(str(x).split(' ')[0], '%Y-%m-%d'))
dfEventsData = dfEventsData.loc[dfEventsData['ReleaseTime'] > min(dfMarketData.index),:]
dfEventsData['ReleaseTime'] = pd.to_datetime(dfEventsData['ReleaseTime'])
dfEventsData = dfEventsData.set_index('ReleaseTime')
dfEventsData = dfEventsData.sort_index()
dfEventsData.tail()

,EventID,Datetime,ActualValue,PreviousValue,Importance,PercentChng,DummyChng,Date
ReleaseTime,,,,,,,,
2023-04-12 11:30:00,840200002,2023.04.12 11:30:00,-0.409,-0.970,2,-0.578351,-1,2023-04-12
2023-04-12 14:00:00,840220014,2023.04.12 14:00:00,3.455,3.985,2,-0.132999,-1,2023-04-12
2023-04-12 15:00:00,840150001,2023.04.12 15:00:00,-378.100,-262.400,2,0.440930,1,2023-04-12
2023-04-13 09:30:00,840140001,2023.04.13 09:30:00,239.000,228.000,2,0.048246,1,2023-04-13
2023-04-13 09:30:00,840030001,2023.04.13 09:30:00,-0.500,-0.100,2,4.000000,1,2023-04-13


In [4]:
from functools import reduce

def get_events_by_datatype(data: str ='ActualValue'):
    if not data in dfEventsData.columns:
        return None
    
    df = reduce(lambda left, right:
                    pd.merge(left , right,
                            on = ['Datetime'],
                            how = "outer"),
                    [pd.DataFrame({'Datetime': dfEventsData[dfEventsData['EventID']==i].index, i: dfEventsData[dfEventsData['EventID']==i][data]}) for i in dfEventsData['EventID'].unique()])

    df=df.merge(dfMarketData['Returns'], on=['Datetime'], how = 'inner')

    df=df.sort_values('Datetime')
    df.reset_index(inplace=True,drop=True)
    df.fillna(method='ffill', inplace=True)
    df.set_index('Datetime',inplace=True)
    # return df[df['Close']>0]
    return df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [5]:
def get_event_name(id: int):
    return dfEventsFromTo[id]['EventName']

In [6]:
df=get_events_by_datatype('PercentChng')
df.tail()

,840030023,840030015,840030016,840030018,840100001,840100002,840030001,840030005,840030006,840200001,...,840190001,840050014,840030029,840030028,840020001,840020003,840040003,840040005,840010015,Returns
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-04-05 09:30:00,-0.401806,0.058824,-0.398453,-0.333333,-0.001686,-0.003958,-1.142857,-0.2,0.25,-7.704566,...,-0.400826,0.052632,1.909091,-0.433333,0.03289,-0.5625,-0.001812,0.08,-0.047447,-0.000492
2023-04-05 10:45:00,-0.401806,0.058824,-0.398453,-0.333333,-0.001686,-0.003958,-1.142857,-0.2,0.25,-7.704566,...,-0.400826,0.052632,1.909091,-0.433333,0.03289,-0.5625,-0.001812,0.08,-0.047447,-0.000493
2023-04-05 11:00:00,-0.401806,0.058824,-0.398453,-0.333333,-0.001686,-0.003958,-1.142857,-0.2,0.25,-7.704566,...,-0.400826,0.052632,1.909091,-0.433333,0.03289,-0.5625,-0.070780,-0.05,-0.047447,-0.002171
2023-04-05 11:30:00,-0.401806,0.058824,-0.398453,-0.333333,-0.001686,-0.003958,-1.142857,-0.2,0.25,-0.500734,...,-0.400826,0.052632,1.909091,-0.433333,0.03289,-0.5625,-0.070780,-0.05,-0.047447,-0.000197
2023-04-06 09:30:00,-0.401806,0.058824,-0.398453,-0.333333,-0.001686,-0.003958,-1.142857,-0.2,0.25,-0.500734,...,-0.400826,0.052632,1.909091,-0.433333,0.03289,-0.5625,-0.070780,-0.05,-0.047447,-0.000197


In [65]:
dates_to_shift=list(np.unique([sorted(list(dfEventsData[dfEventsData['Datetime'] == i]['EventID'])) for i in list(dfEventsData['Datetime'].unique()) if len(sorted(list(dfEventsData[dfEventsData['Datetime'] == i]['EventID'])))>1]))
[dfEventsData[dfEventsData['EventID']==i]['Datetime'] for lists in dates_to_shift for i in lists]

In [141]:
from collections import ChainMap
lista_total=list([{event: sorted(dfEventsData[dfEventsData['EventID']==event]['Datetime'])} for event in list(dfEventsData['EventID'].unique())])
data = dict(ChainMap(*lista_total))
rev_multidict = {}
for key, value in data.items():
    rev_multidict.setdefault(str(value), set()).add(key)
repeated=[list(i) for i in list(rev_multidict.values()) if len(i)>1]
repeated

[[840040003, 840040005],
 [840030028, 840030029],
 [840040001, 840040004],
 [840120003, 840120004],
 [840010001, 840010002, 840010003, 840010004, 840010005],
 [840020013, 840020014],
 [840020008, 840020009],
 [840120001, 840120002],
 [840500002, 840500003],
 [840070001, 840070002],
 [840050026, 840050011],
 [840020016, 840020004, 840020005, 840020015],
 [840020027, 840020028],
 [840020010, 840020011, 840020012],
 [840210001, 840210002],
 [840200001, 840200002],
 [840030005, 840030006],
 [840030016, 840030018, 840030023, 840030015]]

In [21]:
from statsmodels.tsa.stattools import grangercausalitytests

def grangers_causation_matrix(data, variables, test='ssr_chi2test', maxlag=1, verbose=False):
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value <= .05
    df.columns = [get_event_name(var) if var != 'Returns' else var for var in variables]
    df.index = [get_event_name(var) if var != 'Returns' else var for var in variables]
    return df

In [22]:
df_to_test = df.dropna()
df_granger=grangers_causation_matrix(df_to_test, list(df_to_test.columns))

In [23]:
df_granger.head()

,Relatório de Emprego - Payroll - Privado,Taxa de Desemprego,Relatório de Emprego (Payroll) não-agrícola,Ganho Médio por Hora Trabalhada (Mensal),Contagem de Sondas Baker Hughes,Contagem Total de Sondas dos EUA por Baker Hughes,Índice de Preços ao Produtor (Mensal),Núcleo do Índice de Preços ao Consumidor (IPC) (Mensal),Núcleo do Índice de Preços ao Consumidor (IPC) (Mensal),EIA: Estoques de Petróleo Bruto,...,Variação de Empregos Privados ADP,Taxa-alvo de Fundos da Reserva Federal (Fed),Custo Unitário da Mão de Obra (Trimestral),Produtividade do Setor Não Agrícola (Trimestral),Balança Comercial,Encomendas à Indústria (Mensal),Índice de gerentes de compras não-manufatureiros (PMI) do ISM,ISM Não-Manufatura: Emprego,Transações Correntes,Returns
Relatório de Emprego - Payroll - Privado,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Taxa de Desemprego,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,True
Relatório de Emprego (Payroll) não-agrícola,True,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
Ganho Médio por Hora Trabalhada (Mensal),True,True,True,False,False,False,True,False,True,False,...,False,False,False,False,False,False,False,False,False,True
Contagem de Sondas Baker Hughes,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [24]:
[dict({'y': idx, 'x':[col for col in df_granger.columns if (df_granger.loc[idx][col].all()).all()]}) for idx in list(df_granger.index)]

[{'y': 'Relatório de Emprego - Payroll - Privado',
  'x': ['Taxa de Desemprego',
   'Relatório de Emprego (Payroll) não-agrícola',
   'Gastos Pessoais (Mensal)',
   'Vendas Pendentes de Moradias (Mensal)']},
 {'y': 'Taxa de Desemprego',
  'x': ['Leilão Note a 10 anos',
   'Pedidos Iniciais de Subsídio de Desemprego',
   'Índice Empire State de Atividade Industrial do Fed de Nova York',
   'Estoques das Empresas',
   'PMI Composto S&P Global',
   'Índice de Gerentes de Compras (PMI) no Setor de Serviços da S&P Global',
   'Vendas de Casas Usadas (Mensal)',
   'Índice do Instituto de Gestão de Provisões (ISM) de Emprego no Setor Manufatureiro',
   'Índice de Gestores de Compras (PMI) no setor industrial do ISM',
   'Balança Comercial',
   'ISM Não-Manufatura: Emprego',
   'Returns']},
 {'y': 'Relatório de Emprego (Payroll) não-agrícola',
  'x': ['Relatório de Emprego - Payroll - Privado',
   'Taxa de Desemprego',
   'Ganho Médio por Hora Trabalhada (Mensal)',
   'Leilão Note a 10 anos',
